<a href="https://colab.research.google.com/github/minchillo4/factor-invest/blob/main/portproj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Análise Quantitativa de Ações B3 📊

O objetivo desse notebook é montar um modelo simplificado de investimento por fatores.


Primeiro, vamos instalar e importar as bibliotecas necessárias


In [25]:
!pip install fundamentus
!pip install yfinance
!pip install vaderSentiment
!pip install googletrans==4.0.0-rc1


!pip install --upgrade deepl

  Using cached googletrans-4.0.0rc1-py3-none-any.whl
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.1.0a0
    Uninstalling googletrans-3.1.0a0:
      Successfully uninstalled googletrans-3.1.0a0


In [2]:
import fundamentus
import pandas as pd
import numpy as np
import yfinance as yf

Agora, vamos extrair indicadores fundamentalistas de ações da B3.

In [3]:
df = fundamentus.get_resultado_raw()
df

Multiples,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
papel,,,,,,,,,,,,,,,,,,,,
AALR3,8.77,-3.94,1.02,0.899,0.0000,0.377,-4.64,48.62,-0.96,84.09,12.98,0.0185,-0.2210,0.74,0.0091,-0.2592,1497420.0,1.015110e+09,1.04,0.0354
ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.1429,0.0,5.721810e+09,0.00,0.7094
ABCB4,23.42,6.79,0.97,0.000,0.0643,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.1429,14023900.0,5.721810e+09,0.00,0.7094
ABEV3,13.43,14.03,2.34,2.566,0.0544,1.534,57.08,12.44,-24.04,11.62,8.55,0.2063,0.1882,1.11,0.1650,0.1671,302139000.0,9.025490e+10,0.04,0.1323
ABYA3,4.91,-214.80,1.76,2.055,0.0000,0.527,1.98,19.96,-2.75,33.67,33.67,0.1029,-0.0096,2.09,0.0278,-0.0082,0.0,2.920600e+08,1.31,0.1641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WLMM4,27.00,9.31,1.47,0.465,0.0332,1.191,2.72,7.70,3.19,6.99,6.75,0.0603,0.0499,4.54,0.1834,0.1577,13707.8,6.698980e+08,0.01,0.2265
WMBY3,25.39,-19.30,2.87,0.836,0.0000,0.182,1.20,8.62,-1.50,25.02,25.02,0.0970,-0.0705,1.44,0.0239,-0.1486,0.0,2.124390e+08,6.51,-0.1448
WSON33,67.00,8.07,0.98,1.067,0.0000,0.400,13.41,2.42,-0.89,5.08,5.08,0.4414,0.1358,1.26,0.1836,0.1217,0.0,2.148530e+09,1.21,0.0523


In [4]:
print(df.columns)

Index(['Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo', 'P/Cap.Giro',
       'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA', 'Mrg Ebit',
       'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses', 'Patrim. Líq',
       'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object', name='Multiples')


# Entendendo os dados...
Observe que no dataframe, cada coluna do dataframe corresponde a um indicador diferente e o índice é o "ticker" da ação.

Agora, vamos entender o que são os indicadores que acabamos de extrair (não se apegue ao nome das categorias dos indicadores).




## Valuation ⚖


*   **P/L (Price to Earnings):** Relação entre o preço da ação e o lucro por ação. É uma medida da avaliação de uma empresa com base em seu lucro.
P/VP (Price to Book): Relação entre o preço da ação e o valor patrimonial por ação. É uma medida da avaliação de uma empresa com base em seu patrimônio líquido.
*   **PSR (Price to Sales)**: Relação entre o preço da ação e as vendas por ação. É uma medida da avaliação de uma empresa com base em suas vendas.
* **P/VP (Price to Book):** Relação entre o preço da ação e o valor patrimonial por ação. É uma medida da avaliação de uma empresa com base em seu patrimônio líquido.
* **Div.Yield (Dividend Yield):** Dividendo por ação dividido pelo preço da ação. É uma medida da rentabilidade de um investimento em ações.
* **P/L (Price to Earnings):** Relação entre o preço da ação e o lucro por ação. É uma medida da avaliação de uma empresa com base em seu lucro.





## Eficiência e Rentabilidade ⚡

* **P/Ativo (Price to Asset):** Relação entre o preço da ação e o valor do ativo total. É uma medida da avaliação de uma empresa com base em seus ativos.
* **P/Cap.Giro (Price to Working Capital):** Relação entre o preço da ação e o capital de giro. É uma medida da avaliação de uma empresa com base em sua liquidez.
* **P/EBIT (Price to EBIT):** Relação entre o preço da ação e o lucro antes de juros, impostos, depreciação e amortização. É uma medida da avaliação de uma empresa com base em sua rentabilidade operacional.
* **P/Ativ Circ.Liq (Price to Current Assets):** Relação entre o preço da ação e os ativos circulantes líquidos. É uma medida da avaliação de uma empresa com base em sua liquidez.
Renda Fixa e Endividamento

* **Dív.Brut/ Patrim. (Dívida Bruta/Patrimônio Líquido):** Relação entre a dívida bruta e o patrimônio líquido. É uma medida da alavancagem financeira de uma empresa.

## Rentabilidade e Crescimento 🔼

* **ROIC (Return on Invested Capital):** Retorno sobre o capital investido. É uma medida da rentabilidade de uma empresa com base em seus ativos.
* **ROE (Return on Equity):** Retorno sobre o patrimônio líquido. É uma medida da rentabilidade de uma empresa com base em seu capital próprio.
* **Cresc. Rec.5a (Crescimento Recente):** Crescimento das vendas nos últimos 5 anos. É uma medida do potencial de crescimento de uma empresa.
Liquidez e Solidez Financeira

* **Liq. Corr. (Liquidez Corrente):** Relação entre os ativos circulantes e os passivos circulantes. É uma medida da liquidez de uma empresa.
* **Liq.2meses (Liquidez 2 meses):** Relação entre os ativos circulantes e os passivos circulantes com vencimento em até 2 meses. É uma medida da liquidez de curto prazo de uma empresa.

## Margens e Eficiência Operacional 🔨

* **Mrg Ebit (Margem EBIT):** Margem EBIT. É uma medida da eficiência operacional de uma empresa.
Mrg. Líq. (Margem Líquida):** Margem líquida. É uma medida da rentabilidade de uma empresa após a dedução de todos os custos e despesas.
Valuation baseado em Fluxo de Caixa

* **EV/EBIT (Enterprise Value to EBIT):** Valor da empresa dividido pelo EBIT. É uma medida da avaliação de uma empresa com base em seu fluxo de caixa operacional.
* **EV/EBITDA (Enterprise Value to EBITDA):** Valor da empresa dividido pelo EBITDA. É uma medida da avaliação de uma empresa com base em seu fluxo de caixa operacional antes da dedução da amortização e da depreciação.
* **Mg. Liq (Margem Líquida):** A margem líquida é o indicador financeiro que revela a porcentagem de lucro em relação às receitas que uma empresa apresentou no seu demonstrativo de resultados trimestrais ou em seu consolidado anual.



In [5]:
df.head(5)

Multiples,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
papel,,,,,,,,,,,,,,,,,,,,
AALR3,8.77,-3.94,1.02,0.899,0.0000,0.377,-4.64,48.62,-0.96,84.09,12.98,0.0185,-0.2210,0.74,0.0091,-0.2592,1497420.0,1.015110e+09,1.04,0.0354
ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.1429,0.0,5.721810e+09,0.00,0.7094
ABCB4,23.42,6.79,0.97,0.000,0.0643,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.1429,14023900.0,5.721810e+09,0.00,0.7094
ABEV3,13.43,14.03,2.34,2.566,0.0544,1.534,57.08,12.44,-24.04,11.62,8.55,0.2063,0.1882,1.11,0.1650,0.1671,302139000.0,9.025490e+10,0.04,0.1323
ABYA3,4.91,-214.80,1.76,2.055,0.0000,0.527,1.98,19.96,-2.75,33.67,33.67,0.1029,-0.0096,2.09,0.0278,-0.0082,0.0,2.920600e+08,1.31,0.1641


Vamos agora criar uma lista para armazenar as ações que analisaremos.

In [6]:
l_acoes = []

In [7]:
from datetime import datetime, timedelta

def is_traded_recently(acao):
  try:
    stock = yf.Ticker(acao)
    historical_data = stock.history(period='1mo')

    last_trading_day = historical_data.index[-1].date()

    today = datetime.now().date()
    window_start = today - timedelta(days=5)
    return window_start <= last_trading_day <= today
  except Exception:
    return False


In [8]:
is_traded_recently('PETR4.SA')

True

In [9]:
df['ACAO'] = df.index + '.SA'
df

Multiples,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,...,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,ACAO
papel,,,,,,,,,,,,,,,,,,,,,
AALR3,8.77,-3.94,1.02,0.899,0.0000,0.377,-4.64,48.62,-0.96,84.09,...,0.0185,-0.2210,0.74,0.0091,-0.2592,1497420.0,1.015110e+09,1.04,0.0354,AALR3.SA
ABCB3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,...,0.0000,0.0000,0.00,0.0000,0.1429,0.0,5.721810e+09,0.00,0.7094,ABCB3.SA
ABCB4,23.42,6.79,0.97,0.000,0.0643,0.000,0.00,0.00,0.00,0.00,...,0.0000,0.0000,0.00,0.0000,0.1429,14023900.0,5.721810e+09,0.00,0.7094,ABCB4.SA
ABEV3,13.43,14.03,2.34,2.566,0.0544,1.534,57.08,12.44,-24.04,11.62,...,0.2063,0.1882,1.11,0.1650,0.1671,302139000.0,9.025490e+10,0.04,0.1323,ABEV3.SA
ABYA3,4.91,-214.80,1.76,2.055,0.0000,0.527,1.98,19.96,-2.75,33.67,...,0.1029,-0.0096,2.09,0.0278,-0.0082,0.0,2.920600e+08,1.31,0.1641,ABYA3.SA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WLMM4,27.00,9.31,1.47,0.465,0.0332,1.191,2.72,7.70,3.19,6.99,...,0.0603,0.0499,4.54,0.1834,0.1577,13707.8,6.698980e+08,0.01,0.2265,WLMM4.SA
WMBY3,25.39,-19.30,2.87,0.836,0.0000,0.182,1.20,8.62,-1.50,25.02,...,0.0970,-0.0705,1.44,0.0239,-0.1486,0.0,2.124390e+08,6.51,-0.1448,WMBY3.SA
WSON33,67.00,8.07,0.98,1.067,0.0000,0.400,13.41,2.42,-0.89,5.08,...,0.4414,0.1358,1.26,0.1836,0.1217,0.0,2.148530e+09,1.21,0.0523,WSON33.SA


In [10]:
 df['NEGOCIADA'] = df['ACAO'].apply(is_traded_recently)

ERROR:yfinance:ABCB3.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:ABYA3.SA: No data found, symbol may be delisted
ERROR:yfinance:ACES3.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:ACES4.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AEDU11.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AEDU3.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AELP3.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AESL3.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AESL4.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AFLU3.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AFLU5.SA: No price data found, symbol may be delisted (period=1mo)
ERROR:yfinance:AGEI3.SA: No data found, symbol may be delisted
ERROR:yfinance:AGEN33.SA: No data found, symbol may b

In [11]:
df = df[df['NEGOCIADA'] == True]
df

Multiples,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,...,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a,ACAO,NEGOCIADA
papel,,,,,,,,,,,,,,,,,,,,,
AALR3,8.77,-3.94,1.02,0.899,0.0000,0.377,-4.64,48.62,-0.96,84.09,...,-0.2210,0.74,0.0091,-0.2592,1.497420e+06,1.015110e+09,1.04,0.0354,AALR3.SA,True
ABCB4,23.42,6.79,0.97,0.000,0.0643,0.000,0.00,0.00,0.00,0.00,...,0.0000,0.00,0.0000,0.1429,1.402390e+07,5.721810e+09,0.00,0.7094,ABCB4.SA,True
ABEV3,13.43,14.03,2.34,2.566,0.0544,1.534,57.08,12.44,-24.04,11.62,...,0.1882,1.11,0.1650,0.1671,3.021390e+08,9.025490e+10,0.04,0.1323,ABEV3.SA,True
ADHM3,1.56,-4.09,-1.29,770.924,0.0000,105.126,-1.43,-5.69,-1.27,-5.69,...,-188.6060,0.00,5.2170,0.3147,2.064000e+03,-1.977600e+07,0.00,0.4348,ADHM3.SA,True
AERI3,0.82,-9.31,1.25,0.316,0.0000,0.267,1.24,3.46,-2.42,6.94,...,-0.0339,1.46,0.1074,-0.1342,5.512650e+06,8.150790e+08,2.17,0.1041,AERI3.SA,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WIZC3,6.39,10.51,2.22,0.973,0.0697,0.445,-17.54,2.23,-1.24,2.45,...,0.1781,0.90,0.2437,0.2118,3.455440e+06,4.592540e+08,0.77,0.1169,WIZC3.SA,True
WLMM3,22.53,7.76,1.22,0.388,0.0362,0.994,2.27,6.43,2.66,5.71,...,0.0499,4.54,0.1834,0.1577,1.679180e+03,6.698980e+08,0.01,0.2265,WLMM3.SA,True
WLMM4,27.00,9.31,1.47,0.465,0.0332,1.191,2.72,7.70,3.19,6.99,...,0.0499,4.54,0.1834,0.1577,1.370780e+04,6.698980e+08,0.01,0.2265,WLMM4.SA,True


In [12]:
df_f = df.drop(['Cotação', 'NEGOCIADA','ACAO','Patrim. Líq','Cresc. Rec.5a'], axis=1)
df_f

Multiples,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Dív.Brut/ Patrim.
papel,,,,,,,,,,,,,,,,,
AALR3,-3.94,1.02,0.899,0.0000,0.377,-4.64,48.62,-0.96,84.09,12.98,0.0185,-0.2210,0.74,0.0091,-0.2592,1.497420e+06,1.04
ABCB4,6.79,0.97,0.000,0.0643,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.1429,1.402390e+07,0.00
ABEV3,14.03,2.34,2.566,0.0544,1.534,57.08,12.44,-24.04,11.62,8.55,0.2063,0.1882,1.11,0.1650,0.1671,3.021390e+08,0.04
ADHM3,-4.09,-1.29,770.924,0.0000,105.126,-1.43,-5.69,-1.27,-5.69,-5.81,-135.4850,-188.6060,0.00,5.2170,0.3147,2.064000e+03,0.00
AERI3,-9.31,1.25,0.316,0.0000,0.267,1.24,3.46,-2.42,6.94,5.88,0.0912,-0.0339,1.46,0.1074,-0.1342,5.512650e+06,2.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WIZC3,10.51,2.22,0.973,0.0697,0.445,-17.54,2.23,-1.24,2.45,1.95,0.4365,0.1781,0.90,0.2437,0.2118,3.455440e+06,0.77
WLMM3,7.76,1.22,0.388,0.0362,0.994,2.27,6.43,2.66,5.71,5.51,0.0603,0.0499,4.54,0.1834,0.1577,1.679180e+03,0.01
WLMM4,9.31,1.47,0.465,0.0332,1.191,2.72,7.70,3.19,6.99,6.75,0.0603,0.0499,4.54,0.1834,0.1577,1.370780e+04,0.01


In [13]:
acao_column = df['ACAO']
acao_column


papel
AALR3    AALR3.SA
ABCB4    ABCB4.SA
ABEV3    ABEV3.SA
ADHM3    ADHM3.SA
AERI3    AERI3.SA
           ...   
WIZC3    WIZC3.SA
WLMM3    WLMM3.SA
WLMM4    WLMM4.SA
YDUQ3    YDUQ3.SA
ZAMP3    ZAMP3.SA
Name: ACAO, Length: 476, dtype: object

In [14]:
from sklearn.preprocessing import MinMaxScaler

In [15]:
scaler=MinMaxScaler(feature_range=(-1, 1))

In [16]:
# Inverse transform to get back the original data
scaled_data = scaler.fit_transform(df_f)

scaled_data


array([[ 0.66831726, -0.98267267, -0.98058527, ...,  0.85498708,
        -0.99810891, -0.33915771],
       [ 0.67932922, -0.9827003 , -0.98141827, ...,  0.86125781,
        -0.98228925, -0.38575269],
       [ 0.68675948, -0.98194309, -0.97904065, ...,  0.86163521,
        -0.61842946, -0.38396057],
       ...,
       [ 0.68191544, -0.98242395, -0.98098741, ...,  0.86148862,
        -0.99998269, -0.38530466],
       [ 0.70743897, -0.98214207, -0.98023131, ...,  0.85993536,
        -0.91416605, -0.31182796],
       [ 0.6581058 , -0.98257318, -0.98103003, ...,  0.85768033,
        -0.98229948, -0.34677419]])

In [17]:
df_acao = pd.DataFrame(acao_column, columns=['ACAO'])

In [18]:
df_acao_scaled = pd.DataFrame(scaled_data, columns=df_f.columns)


In [19]:
import pandas as pd

# Create a DataFrame with the 'ACAO' column
df_acao = pd.DataFrame(acao_column, columns=['ACAO'])

# Add the scaled data to the new DataFrame
df_acao_scaled = pd.DataFrame(scaled_data, columns=df_f.columns)

# Reset the index for both DataFrames
df_acao.reset_index(drop=True, inplace=True)
df_acao_scaled.reset_index(drop=True, inplace=True)

# Concatenate the two DataFrames along the columns axis
result_df = pd.concat([df_acao, df_acao_scaled], axis=1)

# Display the result DataFrame
print(result_df)


         ACAO       P/L      P/VP       PSR  Div.Yield   P/Ativo  P/Cap.Giro  \
0    AALR3.SA  0.668317 -0.982673 -0.980585  -1.000000 -0.992828    0.751361   
1    ABCB4.SA  0.679329 -0.982700 -0.981418  -0.999511 -1.000000    0.751597   
2    ABEV3.SA  0.686759 -0.981943 -0.979041  -0.999586 -0.970816    0.754498   
3    ADHM3.SA  0.668163 -0.983949 -0.267092  -1.000000  1.000000    0.751525   
4    AERI3.SA  0.662806 -0.982546 -0.981125  -1.000000 -0.994920    0.751660   
..        ...       ...       ...       ...        ...       ...         ...   
471  WIZC3.SA  0.683147 -0.982009 -0.980517  -0.999470 -0.991534    0.750706   
472  WLMM3.SA  0.680325 -0.982562 -0.981059  -0.999725 -0.981089    0.751713   
473  WLMM4.SA  0.681915 -0.982424 -0.980987  -0.999748 -0.977341    0.751735   
474  YDUQ3.SA  0.707439 -0.982142 -0.980231  -0.999900 -0.987520    0.751972   
475  ZAMP3.SA  0.658106 -0.982573 -0.981030  -1.000000 -0.992352    0.754654   

       P/EBIT  P/Ativ Circ.Liq   EV/EBI

In [20]:
import requests
import pandas as pd
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AERI3&apikey=REZCZOOESRXAERQ3'
r = requests.get(url)
data = r.json()

print(data)
anla = pd.DataFrame(data)
anla

{'Information': 'Invalid inputs. Please refer to the API documentation https://www.alphavantage.co/documentation#newsapi and try again.'}


ValueError: If using all scalar values, you must pass an index

In [7]:
from pandas import DataFrame
from GoogleNews import GoogleNews
from datetime import date, timedelta
from newspaper import Article
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
from googletrans import Translator
import deepl
import translators as ts

In [4]:
pip install translators --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14578 sha256=df9e46c181828dbcb9b8d0ce8a4eeeefe8c559af12c87d9cbbb0dcd149d27538
  Stored in directory: /root/.cache/pip/wheels/9d/91/30/28e6da53d4f44dc445349b2ffad581968447e4cbc9dd7991b8
Successfully built PyExecJS


In [10]:
def translate_to_english(text):
    translated_text = ts.translate_text(query_text=text, from_language='pt', to_language='en')
    return translated_text

Agora que possuimos os indicadores fundamentalistas vamos criar

In [16]:
now = date.today()
yesterday = date.today() - timedelta(days=30)
google_news = GoogleNews(lang='pt',region='BR')
google_news.search("ITAUSA")
result = google_news.result()
df = DataFrame(result)
df

df['title_english'] = df['title'].apply(translate_to_english)
df

,title,media,date,datetime,desc,link,img,title_english
0,"BTG Pactual recomenda, mais uma vez, compra pa...",Valor Investe,á 1 dia,NaN,,https://valorinveste.globo.com/mercados/renda-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","BTG Pactual recommends, once again, buy for XP..."
1,Itaúsa (ITSA3)(ITSA4) agenda AGO de 2024; saib...,Economia em Pauta,á 5 dias,NaN,,https://economiaempauta.com/itausa-itsa3itsa4-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Itaúsa (ITSA3)(ITSA4) agenda for the 2024 AGM;...
2,Itaúsa (ITSA3)(ITSA4) agenda AGO de 2024; saib...,SpaceMoney,á 5 dias,NaN,,https://www.spacemoney.com.br/investimentos/ac...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Itaúsa (ITSA3)(ITSA4) agenda for the 2024 AGM;...
3,"R$ 1,7 bilhão e venda crucial: O comunicado do...",TV Foco,á 1 semana,NaN,,https://www.otvfoco.com.br/r-17-bilhao-e-venda...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",R$ 1.7 billion and crucial sale: Itaú's statem...
4,Itaúsa (ITSA4) compõe índices ESG da Bolsa em ...,Economia em Pauta,á 1 semana,NaN,,https://economiaempauta.com/itausa-itsa4-compo...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Itaúsa (ITSA4) is part of the Stock Exchange's...
5,Goldman Sachs corta recomendação de Bradesco (...,Inteligência Financeira,á 1 semana,NaN,,https://inteligenciafinanceira.com.br/mercado-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Goldman Sachs cuts Bradesco (BBDC4) recommenda...
6,Goldman Sachs divulga recomendações sobre Itaú...,E-Investidor,á 1 semana,NaN,,https://einvestidor.estadao.com.br/ultimas/ita...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Goldman Sachs releases recommendations on Itaú...
7,"Goldman eleva Itaúsa para compra, corta Brades...",InfoMoney,á 1 semana,NaN,,https://www.infomoney.com.br/mercados/goldman-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","Goldman raises Itaúsa to buy, cuts Bradesco fo..."
8,Itaúsa (ITSA4) volta a integrar outros índices...,Suno,á 1 semana,NaN,,https://www.suno.com.br/noticias/itausa-itsa4-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",Itaúsa (ITSA4) is once again part of other sto...
9,"Bradesco (BBDC4), Itaúsa (ITSA4) e mais 22 pag...",Money Times,á 2 semanas,NaN,,https://www.moneytimes.com.br/bradesco-bbdc4-i...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","Bradesco (BBDC4), Itaúsa (ITSA4) and 22 others..."


In [14]:
translate_to_english('')

'adadsa'

In [55]:
def analyze_sentiment():
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(title)
    return sentiment_scores['compound']



AttributeError: 'NoneType' object has no attribute 'group'

In [24]:

googletrans.version


NameError: name 'googletrans' is not defined